In [30]:
import pika
import time
import random
import traceback
import sys

# =================================================================================================

board = []

for i in range(3):
    board.append(['O'] * 3)


def print_board(board):
    for row in board:
        print(' '.join(row))


def random_row(board):
    return random.randint(1, 3)


def random_col(board):
    return random.randint(1, 3)


ship_row = random_row(board)
ship_col = random_col(board)

print(f'{ship_row} - номер строки коробля игрока 1')
print(f'{ship_col} - номер колонки коробля игрока 1')

# =================================================================================================

conn_params = pika.ConnectionParameters('localhost', '5672')

conn = pika.BlockingConnection(conn_params)

channel_1 = conn.channel()
channel_2 = conn.channel()

channel_1.queue_declare(queue='first-queue')
channel_2.queue_declare(queue='second-queue')


def callback(ch, method, properties, body):
    time.sleep(2)
    guess_row, guess_col = map(int, body.split())
    if guess_row == 10 and guess_col == 10:
        channel_2.stop_consuming()
        return
    print(f'\nХод игрока 2: {guess_row} row, {guess_col} col')
    
    if guess_row == ship_row and guess_col == ship_col:
        print('Игрок 2 победил!')
        channel_1.basic_publish(exchange='',
                        routing_key='first-queue',
                        body='10 10')
        channel_2.stop_consuming()
        return
    else:
        if board[guess_row - 1][guess_col - 1] == 'X':
            print('Выстрел в эту точку уже был произведен!')
        else:
            print('Промах')
            board[guess_row - 1][guess_col - 1] = 'X'
    
    print('Поле игрока 2')
    print_board(board)
    
    time.sleep(2)
    channel_1.basic_publish(exchange='',
                            routing_key='first-queue',
                            body=str(random.randint(1, 3)) + ' ' + str(random.randint(1, 3)))
    time.sleep(2)
    channel_2.basic_consume(on_message_callback=callback,
                            queue='second-queue', auto_ack=True)


channel_1.basic_publish(exchange='',
                        routing_key='first-queue',
                        body=str(random.randint(1, 3)) + ' ' + str(random.randint(1, 3)))

channel_2.basic_consume(on_message_callback=callback,
                        queue='second-queue', auto_ack=True)

try:
    channel_2.start_consuming()
    channel_1.queue_delete(queue='first-queue')
    channel_2.queue_delete(queue='second-queue')
    channel_1.close()
    channel_2.close()
except KeyboardInterrupt:
    channel_2.stop_consuming()
    channel_1.queue_delete(queue='first-queue')
    channel_2.queue_delete(queue='second-queue')
    channel_1.close()
    channel_2.close()
except Exception:
    channel_2.stop_consuming()
    channel_1.queue_delete(queue='first-queue')
    channel_2.queue_delete(queue='second-queue')
    channel_1.close()
    channel_2.close()
    traceback.print_exc(file=sys.stdout)

conn.close()

3 - номер строки коробля игрока 1
1 - номер колонки коробля игрока 1

Ход игрока 2: 2 row, 3 col
Промах
Поле игрока 2
O O O
O O X
O O O

Ход игрока 2: 3 row, 1 col
Игрок 2 победил!


In [4]:
channel_1.queue_delete(queue='first-queue')
channel_2.queue_delete(queue='second-queue')
channel_1.close()
channel_2.close()

In [8]:
import random

In [3]:
board = []

for i in range(5):
    board.append(['O'] * 5)

In [6]:
def print_board(board):
    for row in board:
        print(' '.join(row))

In [7]:
print_board(board)

O O O O O
O O O O O
O O O O O
O O O O O
O O O O O


In [19]:
def random_row(board):
    return random.randint(1, len(board))

def random_col(board):
    return random.randint(1, len(board[0]))

ship_row = random_row(board)
ship_col = random_col(board)

In [20]:
tries = 10

2 2
